In [1]:
import pandas as pd
import numpy as np
import seaborn as sns
import matplotlib.pyplot as plt
from sklearn.tree import DecisionTreeClassifier
from sklearn.discriminant_analysis import QuadraticDiscriminantAnalysis, LinearDiscriminantAnalysis
from sklearn.neighbors import KNeighborsClassifier
from sklearn.ensemble import AdaBoostClassifier, GradientBoostingClassifier, ExtraTreesClassifier, RandomForestClassifier
from sklearn.cross_validation import train_test_split
from sklearn.metrics import confusion_matrix, accuracy_score
from sklearn.ensemble import VotingClassifier

C:\Users\Rowena\Anaconda3\lib\site-packages\sklearn\cross_validation.py:44: DeprecationWarning: This module was deprecated in version 0.18 in favor of the model_selection module into which all the refactored classes and functions are moved. Also note that the interface of the new CV iterators are different from that of this module. This module will be removed in 0.20.
  "This module will be removed in 0.20.", DeprecationWarning)


In [4]:
data = pd.read_csv('Pokemon.csv')

In [5]:
data.head()

,#,Name,Type 1,Type 2,Total,HP,Attack,Defense,Sp. Atk,Sp. Def,Speed,Generation,Legendary
0,1,Bulbasaur,Grass,Poison,318,45,49,49,65,65,45,1,False
1,2,Ivysaur,Grass,Poison,405,60,62,63,80,80,60,1,False
2,3,Venusaur,Grass,Poison,525,80,82,83,100,100,80,1,False
3,3,VenusaurMega Venusaur,Grass,Poison,625,80,100,123,122,120,80,1,False
4,4,Charmander,Fire,NaN,309,39,52,43,60,50,65,1,False


In [6]:
data = data.set_index('#')
data.head()
type1 = data.pop('Type 1')

In [7]:
data.head()

,Name,Type 2,Total,HP,Attack,Defense,Sp. Atk,Sp. Def,Speed,Generation,Legendary
#,,,,,,,,,,,
1,Bulbasaur,Poison,318,45,49,49,65,65,45,1,False
2,Ivysaur,Poison,405,60,62,63,80,80,60,1,False
3,Venusaur,Poison,525,80,82,83,100,100,80,1,False
3,VenusaurMega Venusaur,Poison,625,80,100,123,122,120,80,1,False
4,Charmander,NaN,309,39,52,43,60,50,65,1,False


In [8]:
type1.value_counts()

Water       112
Normal       98
Grass        70
Bug          69
Psychic      57
Fire         52
Rock         44
Electric     44
Ground       32
Ghost        32
Dragon       32
Dark         31
Poison       28
Steel        27
Fighting     27
Ice          24
Fairy        17
Flying        4
Name: Type 1, dtype: int64

In [9]:
# create a binary response variable for each class
Y = pd.get_dummies(type1)
np.sum(Y, axis=0)

Bug          69
Dark         31
Dragon       32
Electric     44
Fairy        17
Fighting     27
Fire         52
Flying        4
Ghost        32
Grass        70
Ground       32
Ice          24
Normal       98
Poison       28
Psychic      57
Rock         44
Steel        27
Water       112
dtype: int64

In [11]:
dummies2list = []
for x in Y:
    word = 'Type_1_' + x
    dummies2list.append(word)
Y.columns = dummies2list

In [12]:
Y.head()

,Type_1_Bug,Type_1_Dark,Type_1_Dragon,Type_1_Electric,Type_1_Fairy,Type_1_Fighting,Type_1_Fire,Type_1_Flying,Type_1_Ghost,Type_1_Grass,Type_1_Ground,Type_1_Ice,Type_1_Normal,Type_1_Poison,Type_1_Psychic,Type_1_Rock,Type_1_Steel,Type_1_Water
#,,,,,,,,,,,,,,,,,,
1,0,0,0,0,0,0,0,0,0,1,0,0,0,0,0,0,0,0
2,0,0,0,0,0,0,0,0,0,1,0,0,0,0,0,0,0,0
3,0,0,0,0,0,0,0,0,0,1,0,0,0,0,0,0,0,0
3,0,0,0,0,0,0,0,0,0,1,0,0,0,0,0,0,0,0
4,0,0,0,0,0,0,1,0,0,0,0,0,0,0,0,0,0,0


#alternative way
Y = pd.DataFrame()
for this_type in types:
    this_Y = (type1 == this_type).astype(int)
    Y['Type_1_' + this_type] = this_Y

In [13]:
data['Generation'].unique()

array([1, 2, 3, 4, 5, 6], dtype=int64)

In [14]:
dummies = pd.get_dummies(data['Generation'])
#dummies2 = pd.get_dummies(data['Type 1'])
dummies3 = pd.get_dummies(data['Type 2'])

dummies1list = []
for x in dummies:
    word = 'Generation_' + str(x)
    dummies1list.append(word)
    
dummies.columns = dummies1list
step_1 = pd.concat([data, dummies], axis=1)  


#dummies2list = []
#for x in dummies2:
#    word = 'Type_1_' + x
#    dummies2list.append(word)
    
dummies3list = []
for x in dummies3:
    word = 'Type_2_' + x
    dummies3list.append(word)
#dummies2.columns = dummies2list
dummies3.columns = dummies3list
final_step = pd.concat([step_1,dummies3], axis = 1)
#data = pd.concat([final_step,dummies3], axis = 1)

#del data['Type 1']
del final_step['Type 2']
del final_step['Generation']
del final_step['Name']
final_step

#Converts the boolean column into a column of 0's and 1's
final_step.Legendary = final_step.Legendary.astype(int)

In [15]:
final_step.head()

,Total,HP,Attack,Defense,Sp. Atk,Sp. Def,Speed,Legendary,Generation_1,Generation_2,...,Type_2_Ghost,Type_2_Grass,Type_2_Ground,Type_2_Ice,Type_2_Normal,Type_2_Poison,Type_2_Psychic,Type_2_Rock,Type_2_Steel,Type_2_Water
#,,,,,,,,,,,,,,,,,,,,,
1,318,45,49,49,65,65,45,0,1,0,...,0,0,0,0,0,1,0,0,0,0
2,405,60,62,63,80,80,60,0,1,0,...,0,0,0,0,0,1,0,0,0,0
3,525,80,82,83,100,100,80,0,1,0,...,0,0,0,0,0,1,0,0,0,0
3,625,80,100,123,122,120,80,0,1,0,...,0,0,0,0,0,1,0,0,0,0
4,309,39,52,43,60,50,65,0,1,0,...,0,0,0,0,0,0,0,0,0,0


In [16]:
final_step.columns

Index(['Total', 'HP', 'Attack', 'Defense', 'Sp. Atk', 'Sp. Def', 'Speed',
       'Legendary', 'Generation_1', 'Generation_2', 'Generation_3',
       'Generation_4', 'Generation_5', 'Generation_6', 'Type_2_Bug',
       'Type_2_Dark', 'Type_2_Dragon', 'Type_2_Electric', 'Type_2_Fairy',
       'Type_2_Fighting', 'Type_2_Fire', 'Type_2_Flying', 'Type_2_Ghost',
       'Type_2_Grass', 'Type_2_Ground', 'Type_2_Ice', 'Type_2_Normal',
       'Type_2_Poison', 'Type_2_Psychic', 'Type_2_Rock', 'Type_2_Steel',
       'Type_2_Water'],
      dtype='object')

In [17]:
X_train, X_test, y_train, y_test = train_test_split( final_step, Y, test_size = 0.3, random_state = 100)

In [18]:
clf_gini = DecisionTreeClassifier(criterion = "gini", random_state = 100,
                               max_depth=3, min_samples_leaf=5)
clf_gini.fit(X_train, y_train['Type_1_Bug'])

DecisionTreeClassifier(class_weight=None, criterion='gini', max_depth=3,
            max_features=None, max_leaf_nodes=None,
            min_impurity_split=1e-07, min_samples_leaf=5,
            min_samples_split=2, min_weight_fraction_leaf=0.0,
            presort=False, random_state=100, splitter='best')

In [19]:
pred = clf_gini.predict(X_test)

In [20]:
confusion_matrix(y_test['Type_1_Bug'],pred)

array([[217,   9],
       [ 12,   2]])

In [21]:
accuracy_score(y_test['Type_1_Bug'],pred)

0.91249999999999998

In [22]:
accuracy_clf_gini = []
confusion_matrices = []
for y_type in y_train:
    clf_gini.fit(X_train, (y_train[y_type]))
    pred = clf_gini.predict(X_test)
    confusion_matrices.append(confusion_matrix(y_test[y_type],pred))
    accuracy_clf_gini.append(accuracy_score(y_test[y_type],pred))

In [23]:
accuracy_clf_gini

[0.91249999999999998,
 0.94999999999999996,
 0.93333333333333335,
 0.9291666666666667,
 0.96666666666666667,
 0.95416666666666672,
 0.9375,
 0.9916666666666667,
 0.98333333333333328,
 0.9375,
 0.95416666666666672,
 0.95416666666666672,
 0.85833333333333328,
 0.96666666666666667,
 0.9291666666666667,
 0.9291666666666667,
 0.95416666666666672,
 0.82499999999999996]

In [24]:
#Gradient Boosting Classifier
GradBoost = GradientBoostingClassifier(n_estimators=350, learning_rate=0.1,
                                 max_depth=1, random_state=0)

In [25]:
accuracy_gradboost = []
for y_type in y_train:
    GradBoost.fit(X_train, y_train[y_type])
    pred = GradBoost.predict(X_test)
    accuracy_gradboost.append(accuracy_score(y_test[y_type],pred))

In [26]:
accuracy_gradboost

[0.94999999999999996,
 0.94999999999999996,
 0.94166666666666665,
 0.93333333333333335,
 0.95833333333333337,
 0.95833333333333337,
 0.9458333333333333,
 0.98750000000000004,
 0.97499999999999998,
 0.93333333333333335,
 0.95416666666666672,
 0.95416666666666672,
 0.85416666666666663,
 0.96666666666666667,
 0.9375,
 0.9291666666666667,
 0.95833333333333337,
 0.8666666666666667]

In [27]:
#Extremely Random Forest
Extratree = ExtraTreesClassifier(max_depth=None,min_samples_split=2)

In [28]:
accuracy_extratree= []
for y_type in y_train:
    Extratree.fit(X_train, y_train[y_type])
    pred = Extratree.predict(X_test)
    accuracy_extratree.append(accuracy_score(y_test[y_type],pred))

In [29]:
accuracy_extratree

[0.92083333333333328,
 0.95416666666666672,
 0.95416666666666672,
 0.92500000000000004,
 0.96250000000000002,
 0.96666666666666667,
 0.9291666666666667,
 0.99583333333333335,
 0.98333333333333328,
 0.91666666666666663,
 0.95416666666666672,
 0.9458333333333333,
 0.86250000000000004,
 0.95833333333333337,
 0.9375,
 0.9291666666666667,
 0.96666666666666667,
 0.85833333333333328]

In [30]:
#Adaptive Boosting
regr = AdaBoostClassifier(learning_rate = 1, n_estimators = 350)

In [31]:
accuracy_regr= []
for y_type in y_train:
    regr.fit(X_train, y_train[y_type])
    pred = regr.predict(X_test)
    accuracy_regr.append(accuracy_score(y_test[y_type],pred))

In [32]:
accuracy_regr

[0.91666666666666663,
 0.93333333333333335,
 0.9375,
 0.92083333333333328,
 0.95833333333333337,
 0.9291666666666667,
 0.91249999999999998,
 0.98750000000000004,
 0.97499999999999998,
 0.89583333333333337,
 0.9291666666666667,
 0.94999999999999996,
 0.86250000000000004,
 0.9458333333333333,
 0.90416666666666667,
 0.89583333333333337,
 0.96250000000000002,
 0.79166666666666663]

In [33]:
finalensem = VotingClassifier(estimators=[('AdaBoost', regr), ('ExtraTrees', Extratree), ('GradientBoost', GradBoost)], voting='soft')

In [34]:
accuracy_finalensem = []
for y_type in y_train:
    finalensem.fit(X_train, y_train[y_type])
    pred = finalensem.predict(X_test)
    accuracy_finalensem.append(accuracy_score(y_test[y_type],pred))

In [35]:
accuracy_finalensem

[0.9291666666666667,
 0.94999999999999996,
 0.9458333333333333,
 0.9375,
 0.96250000000000002,
 0.97499999999999998,
 0.9375,
 0.98750000000000004,
 0.98333333333333328,
 0.9375,
 0.95416666666666672,
 0.95416666666666672,
 0.8666666666666667,
 0.96666666666666667,
 0.93333333333333335,
 0.92500000000000004,
 0.96250000000000002,
 0.87083333333333335]

In [36]:
models = ['clf_gini', 'gradBoost', 'extraTree', 'adaBoost', 'finalEnsem']

In [42]:
accuracy_scores = [accuracy_clf_gini, accuracy_gradboost, accuracy_extratree, accuracy_regr, accuracy_finalensem]

In [55]:
Y.columns

Index(['Type_1_Bug', 'Type_1_Dark', 'Type_1_Dragon', 'Type_1_Electric',
       'Type_1_Fairy', 'Type_1_Fighting', 'Type_1_Fire', 'Type_1_Flying',
       'Type_1_Ghost', 'Type_1_Grass', 'Type_1_Ground', 'Type_1_Ice',
       'Type_1_Normal', 'Type_1_Poison', 'Type_1_Psychic', 'Type_1_Rock',
       'Type_1_Steel', 'Type_1_Water'],
      dtype='object')

In [57]:
accuracy = pd.DataFrame(data = accuracy_scores).transpose()
accuracy.columns = models
accuracy.set_index(Y.columns)

,clf_gini,gradBoost,extraTree,adaBoost,finalEnsem
Type_1_Bug,0.912500,0.950000,0.920833,0.916667,0.929167
Type_1_Dark,0.950000,0.950000,0.954167,0.933333,0.950000
Type_1_Dragon,0.933333,0.941667,0.954167,0.937500,0.945833
Type_1_Electric,0.929167,0.933333,0.925000,0.920833,0.937500
Type_1_Fairy,0.966667,0.958333,0.962500,0.958333,0.962500
Type_1_Fighting,0.954167,0.958333,0.966667,0.929167,0.975000
Type_1_Fire,0.937500,0.945833,0.929167,0.912500,0.937500
Type_1_Flying,0.991667,0.987500,0.995833,0.987500,0.987500
Type_1_Ghost,0.983333,0.975000,0.983333,0.975000,0.983333
Type_1_Grass,0.937500,0.933333,0.916667,0.895833,0.937500


In [58]:
accuracy.describe()

,clf_gini,gradBoost,extraTree,adaBoost,finalEnsem
count,18.000000,18.000000,18.000000,18.000000,18.000000
mean,0.937037,0.941898,0.940046,0.922685,0.943287
std,0.040420,0.033194,0.035932,0.044859,0.032531
min,0.825000,0.854167,0.858333,0.791667,0.866667
25%,0.929167,0.934375,0.926042,0.906250,0.934375
50%,0.943750,0.950000,0.950000,0.929167,0.947917
75%,0.954167,0.958333,0.961458,0.948958,0.962500
max,0.991667,0.987500,0.995833,0.987500,0.987500
